In [59]:
from train.model import Net
import os
from train.puzzle_dataset import PuzzleDataset
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net()
model.to(device)

Net(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=3072, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=8, bias=True)
)

In [61]:
summary(model, (3, 384, 384))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 384, 384]              84
         MaxPool2d-2          [-1, 3, 192, 192]               0
           Dropout-3          [-1, 3, 192, 192]               0
            Linear-4                  [-1, 256]         786,688
            Linear-5                    [-1, 8]           2,056
Total params: 788,828
Trainable params: 788,828
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.69
Forward/backward pass size (MB): 5.06
Params size (MB): 3.01
Estimated Total Size (MB): 9.76
----------------------------------------------------------------


In [62]:
epoch_num = 1
batch_size = 32
train_data = PuzzleDataset("../train-data")
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
loss_func = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)

In [63]:
for epoch in range(epoch_num):
    train_loss = 0
    model.train()
    for batch_image, batch_label in train_loader:
        inputs, labels = batch_image.to(device), batch_label.to(device)
        optimizer.zero_grad()  # Clear optimizers
        output = model.forward(inputs)
        print(inputs.shape, labels.shape, output.shape)
        # Forward pass
        loss = loss_func(output, labels)  # Loss
        loss.backward()  # Calculate gradients (backpropogation)
        optimizer.step()  # Adjust parameters based on gradients
        train_loss += loss.item() * inputs.size(0)  # Add the loss to the training set's rnning loss

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))


torch.Size([32, 3, 384, 384]) torch.Size([32, 8]) torch.Size([1152, 8])


RuntimeError: The size of tensor a (1152) must match the size of tensor b (32) at non-singleton dimension 0